In [6]:
import torch
import numpy as np
import open3d as o3d
from tqdm import tqdm
import point_cloud_utils as pcu
def farthest_point_sample(point, npoint=2048):
    """
    Input:
        xyz: pointcloud data, [N, D]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [npoint, D]
    """
    N, D = point.shape
    xyz = point[:,:3]
    centroids = np.zeros((npoint,))
    distance = np.ones((N,)) * 1e10
    farthest = np.random.randint(0, N)
    for i in range(npoint):
        centroids[i] = farthest
        centroid = xyz[farthest, :]
        dist = np.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = np.argmax(distance, -1)
    return centroids.astype(np.int32)

In [7]:
import os
d = '/mnt/d/junch_data/cityu_demo/text_pc'
for fp in os.listdir(d):
    
    pcd=o3d.io.read_point_cloud(os.path.join(d, fp))
    p=np.asarray(pcd.points)
    idxes = farthest_point_sample(point=p)
    p=np.asarray(pcd.points)[idxes]
    c=np.asarray(pcd.colors)[idxes]
    s_d = '/mnt/d/junch_data/cityu_demo/text_pc'
    save_path = os.path.join(s_d , fp.split('.ply')[0]+'.xyz')
    with open(save_path, 'w') as file:
        for point, color in zip(p, c):
            # 创建一行，包含坐标和颜色信息
            line = f"{point[0]} {point[1]} {point[2]}\n"
            # 写入文件
            file.write(line)
    # s_p=os.path.join('/mnt/d/junch_data/diffusion_231019/all_chair_pointcloud_vis',fp+'.ply')
    # save_pc_as_ply(s_p,p,c)

In [8]:
from plyfile import PlyData, PlyElement
def write_ply_with_colors(vertices, colors,save_path,  text=True):
    """
    save_path : path to save: '/yy/XX.ply'
    vertices: array of point cloud vertices of size (N, 3)
    colors: array of colors corresponding to vertices of size (N, 3), in normalized [0, 1] range
    """
    # Ensure that vertices and colors have the same number of points
    # print(type(colors))
    # print(type(colors[0]))

    assert vertices.shape[0] == colors.shape[0], "Vertices and colors must have the same number of points"

    # Normalize color values to [0, 255] range
    colors = (colors).astype(np.uint8)

    # Combine vertices and colors into one array
    vertex_colored = np.zeros(vertices.shape[0], dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
                                                        ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])

    for i in range(vertices.shape[0]):
        vertex_colored[i] = tuple(vertices[i]) + tuple(colors[i])

    # Define the PlyElement for vertices with colors and create the PlyData instance
    ply_element = PlyElement.describe(vertex_colored, 'vertex', comments=['vertices with color'])
    ply_data = PlyData([ply_element], text=text)

    # Write the data to a .ply file
    ply_data.write(save_path)
    
import numpy as np

def find_nearest_point(arr1, arr2):
    # arr1: [8192, 3], arr2: [4096, 3]
    
    # 计算距离矩阵，shape 为 [8192, 4096]
    distances = np.sqrt(((arr1[:, np.newaxis, :] - arr2[np.newaxis, :, :]) ** 2).sum(axis=2))
    
    # 找到最近点的索引
    min_indices = np.argmin(distances, axis=1)
    
    return min_indices

# 示例使用
# 假设 arr1 和 arr2 是已经创建好的 numpy 数组
# arr1 = np.random.rand(8192, 3)
# arr2 = np.random.rand(4096, 3)
# min_indices = find_nearest_point(arr1, arr2)
def read_point_cloud(file_path):
    with open(file_path, 'r') as file:
        ply_data = file.readlines()
    
    # Find where the header ends (indicated by "end_header")
    header_end_index = 0
    for i, line in enumerate(ply_data):
        if "end_header" in line:
            header_end_index = i + 1
            break
    
    # Process the vertex data after the header
    vertex_data = []
    for line in ply_data[header_end_index:]:
        if line.strip() == "":  # Skip empty lines if any
            continue
        parts = line.split()
        if len(parts) == 6:  # x, y, z, red, green, blue
            # Convert the vertex data to float for coordinates and int for colors
            vertex = [float(parts[0]), float(parts[1]), float(parts[2]),
                      int(parts[3]), int(parts[4]), int(parts[5])]
            vertex_data.append(vertex)
    
    # Convert the list to a NumPy array
    vertex_array = np.array(vertex_data)
    
    return vertex_array
import numpy as np

def read_xyz_file(file_path):
    """
    读取 XYZ 文件并返回一个 NumPy 数组，其中包含坐标数据。
    
    参数:
    file_path -- XYZ 文件的路径
    
    返回:
    coords -- 坐标的 NumPy 数组
    """
    # 使用 NumPy 的 loadtxt 方法直接读取坐标数据
    coords = np.loadtxt(file_path)
    return coords

In [ ]:
gradpu_d = '/mnt/d/junch_data/Grad-PU/Grad-PU/pretrained_model/pugan/test/4X'
for f_p in os.listdir(gradpu_d):
    arr1 = read_xyz_file(os.path.join(gradpu_d, f_p))
    arr2_fp = os.path.join('/mnt/d/junch_data/cityu_demo/text_pc', f_p.split('.xyz')[0]+'.ply')
    arr2 = read_point_cloud(arr2_fp)
    arr2_points = arr2[:,:3]
    arr2_colors = arr2[:,3:]
    indices = find_nearest_point(arr1,arr2_points)
    s_p = os.path.join('/mnt/d/junch_data/cityu_demo/text_pc', 'textpc_gradpu_color.ply')
    write_ply_with_colors(vertices=arr1, colors=arr2_colors[indices], save_path=s_p)